In [1]:
# load credentials from environment variables
%load_ext dotenv
%dotenv

# util
import boto3

# date & time
from datetime import date, datetime
from util import date_interval_endpoints as endpoints

# Oauth
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

## Get authorization token

In [2]:
# Your client credentials
client_id = %env SH_CLIENT_ID
client_secret = %env SH_CLIENT_SECRET

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                          client_id=client_id, client_secret=client_secret)

resp = oauth.get("https://services.sentinel-hub.com/oauth/tokeninfo")

## Configure request (evalscript)

Enter start and end date, input bands, indices. The resulting files will have two time intervals per month, being split at `day_of_new_interval`.

In [3]:
startdate = date(2017,9,1) # Y,M,D
enddate = date(2018,11,30)  # Y,M,D

input_bands = [
    "B02",
    "B03",
    "B04",
    "B05",
    "B06",
    "B07",
    "B08",
    "B8A",
    "B11",
    "B12"
]


indices = [
    "NDVI",
    "GNDVI",
    "BNDVI",
    "CVI",
    "NDSI",
    "NDWI"
]

bucket_name = "eox-masterdatacube"

day_of_new_interval = 16 # leave this unchanged in most of the cases

### Calculate parameters

In [4]:
eps = endpoints(startdate, enddate, day_of_new_interval)

timestamps  = [int(d.timestamp()) for d in eps]   # timestamps for arithmetic
avg_times = [(left+right)/2 for left,right in zip(timestamps[::2],timestamps[1::2])]
avg_times = [datetime.utcfromtimestamp(a) for a in avg_times]
avg_times = [dt.isoformat() for dt in avg_times]

In [5]:
masks = ["SCL", "dataMask"] # SCL ... Scene Classification Layer

output_bands = input_bands + indices
output_array =  [ { 'id': "\"" + ob + "\"", 'bands': len(avg_times), "sampleType": "SampleType.UINT16"} for ob in output_bands ]
for oa in output_array:
    if oa["id"] == '"CVI"':
        oa["sampleType"] = "SampleType.FLOAT32"
output_array = str(output_array).replace("'", '')

int_bands = '{' + ','.join([f'{ib}: []' for ib in input_bands]) + '}'
results_object = '{' + ','.join([f'{ob}: []' for ob in output_bands]) + '}'
debug_results = '{' + ','.join([f"{output_bands[i]}: [{i+1}]" for i in range(len(output_bands))]) + '}'
responses = [{"identifier": ob,"format": {"type": "image/tiff"}} for ob in output_bands]

### Evalscript & Payload

In [6]:
with open('evalscript.js', 'r') as file:
    evalscript = file.read()
    
evalscript = evalscript.format(
    bands              =str(input_bands+masks),
    output_array       =output_array,
    results_object     =results_object,
    day_of_new_interval=day_of_new_interval,
    enddate_unix       =datetime(*enddate.timetuple()[:3],23,59,59).timestamp()*1000,
    debug_results      =debug_results,
    int_bands          =int_bands,
    indices            =indices,
    avg_times          =avg_times
)

In [7]:
mab_geometry = { "type": "Polygon", "coordinates": 
    [
        [
            [ 14.249086675661697, 48.549686576488739, 0.0 ],
            [ 14.804694846753847, 48.546347802635076, 0.0 ],
            [ 14.804740209647099, 48.662420784507489, 0.0 ],
            [ 14.802209327231056, 48.752811114695149, 0.0 ],
            [ 16.224291241591107, 48.74506103134155, 0.0 ],
            [ 16.19780424032631, 47.621347235969026, 0.0 ],
            [ 14.263435524016538, 47.627863161515464, 0.0 ],
            [ 14.249086675661697, 48.549686576488739, 0.0 ]
        ]
    ]
}

payload = {
  "processRequest": {
    "input": {
      "bounds": {
        "properties": {
          "crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"
        },
        "geometry": mab_geometry
      },
      "data": [
        {
          "location": "AWS:eu-central-1",
          "type": "S2L2A",
          "dataFilter": {
            "timeRange": {
              "from": eps[0].isoformat() + 'Z', # starttime
              "to": eps[-1].isoformat() + 'Z'   # endtime
            },
            "mosaickingOrder": "mostRecent",
            "maxCloudCoverage": 100,
            "previewMode": "DETAIL"
          }
        }
      ]
    },
    "output": {
      "responses": [*responses]
    },
    "evalscript": evalscript
  },
  "tilingGridId": 0,
  "bucketName": bucket_name,
  "resolution": 10.0,
  "description": f"Test MAB"
}

headers = {
  #'Accept': 'application/tar'
}

## Send request

In [8]:
url = 'https://services.sentinel-hub.com/batch/v1/process/'

response = oauth.request("POST", url, headers=headers, json = payload).json()
request_id = response["id"]
vE = response["valueEstimate"]
print(f"""Status of request {request_id}: {response['status']}
Value Estimate: {vE:.3f}""")

oauth.request("POST", f'{url}{request_id}/start')
print('Processing started.')

s3 = boto3.resource('s3')
bk = s3.Bucket(bucket_name)

bk.put_object(Key=request_id + '/userdata.json', Body=json.dumps({
    'bands': output_bands,
    'request_id': request_id,
#     'tiles': tiles,
    'time': avg_times
}))
print('Metadata saved to bucket')

Status of request 787be67b-696a-4853-9a93-0366449ef1df: CREATED
Value Estimate: 492111.426
Processing started.
Metadata saved to bucket
